In [ ]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the current directory to the Python path
if current_dir not in sys.path:
    sys.path.append(current_dir)

# Verify the path
print(sys.path)

In [ ]:
print(sys.path)

In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer

In [ ]:
# Create an instance of PTBTokenizer
tokenizer = PTBTokenizer()

# Prepare your captions
captions_for_image = {
    0: [{"caption": "man plays guitar"}],
    1: [{"caption": "guitar being played by man"}, {"caption": "man holds guitar"}],
}
# Use the tokenize function
tokenized_captions = tokenizer.tokenize(captions_for_image)

print(tokenized_captions)

In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.cider.cider import Cider


class SimpleCIDErEval:
    def __init__(self):
        self.tokenizer = PTBTokenizer()
        self.cider_scorer = Cider()

    def evaluate(self, predictions, references):
        # Format the input for the tokenizer
        gts = {i: [{"caption": c} for c in refs] for i, refs in enumerate(references)}
        res = {i: [{"caption": p}] for i, p in enumerate(predictions)}

        # Tokenize
        gts_tokenized = self.tokenizer.tokenize(gts)
        res_tokenized = self.tokenizer.tokenize(res)

        # Compute CIDEr score
        score, scores = self.cider_scorer.compute_score(gts_tokenized, res_tokenized)

        return score, scores

In [ ]:
import json
from datasets import COCODataset

coco_dataset = COCODataset(
    ann_file="./data/coco/annotations/captions_val2017.json",
    img_dir="./data/coco/val2017",
)

f = open("coco_results.json")
results = json.load(f)
f.close()

candidates = [result["caption"] for result in results]
references = [coco_dataset.get_captions(result["image_id"]) for result in results]

In [ ]:
# Create evaluator
evaluator = SimpleCIDErEval()

overall_score, individual_scores = evaluator.evaluate(candidates, references)

print(f"Overall CIDEr score: {overall_score}")
print("Individual CIDEr scores:")
for i, (preds, refs, scores) in enumerate(zip(predictions, references, individual_scores)):
    print(f"\nImage {i}:")
    print(f"  Predictions: {preds}")
    print(f"  References: {refs}")
    print(f"  Scores: {scores}")

In [ ]:
from pycocoevalcap.tokenizer.ptbtokenizer import PTBTokenizer
from pycocoevalcap.cider.cider import Cider


class SimpleCIDErEval:
    def __init__(self):
        self.tokenizer = PTBTokenizer()
        self.cider_scorer = Cider()

    def evaluate(self, predictions, references):
        # Format the input for the tokenizer
        gts = {i: [{"caption": c} for c in refs] for i, refs in enumerate(references)}
        res = {i: [{"caption": p}] for i, p in enumerate(predictions)}

        # Tokenize
        gts_tokenized = self.tokenizer.tokenize(gts)
        res_tokenized = self.tokenizer.tokenize(res)

        # Compute CIDEr score
        score, scores = self.cider_scorer.compute_score(gts_tokenized, res_tokenized)

        return score, scores


# Example usage
if __name__ == "__main__":
    # Example data
    predictions = ["a man playing guitar", "a woman singing"]
    references = [
        ["man plays guitar", "guitarist performing"],
        ["woman sings into microphone", "singer on stage", "vocal performance"],
    ]

    # Create evaluator
    evaluator = SimpleCIDErEval()

    # Compute scores
    overall_score, individual_scores = evaluator.evaluate(predictions, references)

    # Print results
    print(f"Overall CIDEr score: {overall_score}")
    print("Individual CIDEr scores:")
    for i, (pred, refs, score) in enumerate(zip(predictions, references, individual_scores)):
        print(f"\nImage {i}:")
        print(f"  Prediction: {pred}")
        print(f"  References: {refs}")
        print(f"  Score: {score}")